# Monkey Finger Movement Dataset

## 1. Overview

The following dataset includes data from two recording sessions of a macque performing a two degree-of-freedom (DOF) finger movement task, with neural activity recorded from the hand area of the primary motor cortex. The data presented here, provided by the Chestek Lab at the University of Michigan, is a modified version of data hosted on Deep Blue Data, provided by the University of Michigan Library [here](https://deepblue.lib.umich.edu/data/concern/data_sets/0g354f51t) (also accessible through the Chestek Lab website [here](https://chestekresearch.engin.umich.edu/data-and-resources/)). Specifically, the datasets are taken from Data.zip/MonkeyN_CO.mat and Data.zip/MonkeyN_R.mat. 

The full, unmodified dataset was used to generate many of the results presented in the [Nason, S. R. et al. 2021 study](https://doi.org/10.1016/j.neuron.2021.08.009), "Real-time linear prediction of simultaneous and independent movements of two finger groups using an intracortical brain-machine interface", published in Neuron.

In [1]:
from IPython.display import display, Image

### 1.1 Task

In the finger movement task, a rhesus macaque (Monkey N) performed successive 'trials' where he had to independently move his index finger (IDX) and middle-ring-pinky fingers (MRP) to match target positions within a set time limit. The targets were shown using a monitor placed in front of him, as shown above, which displayed a virtual hand and two spherical targets (red for IDX, yellow for MRP). Once the monkey reached the targets, he had to hold his fingers in place for a set 'hold time' to complete the the trial successfully. After each successful trial, the monkey received a juice reward.

The datasets provided here consist of two different 'target styles', center-out and random targets. In the center-out task, trials alternate between 'rest' trials and 'reach' trials. Rest trials present both targets at halfway between flexion and extension, while reach trials involve some kinda of movement away from rest - split movements in different directions, both fingers flexing/extending, and single group flexion/extension. In the random task, targets are presented pseudo-randomly for both finger groups, although a limit is imposed on how far finger groups can be separated from one another. Datasets from the center-out and random tasks are labelled 'co_feats##.pkl' and 'r_feats##.pkl', respectively.

### 1.2 Data

For these datasets, 96 channels of neural activity were recorded from a Utah array implanted in the hand area of a monkey's motor cortex during the task. The neural feature used in these datasets is Spiking-Band Power (SBP), as described by Nason et al. 2020 in Nature Biomedical engineering ([link](https://doi.org/10.1038/s41551-020-0591-0)).

Additionally, relative finger position along the arc of flexion/extension was measured using a flex sensors in the manipulandum where the monkeys hand was placed. We estimated the finger group velocities and accelerations using the recorded hand positions.

The random and center-out trials provided are each two sets of experiments combined into one (four experiments combined into two total). All trials were collected on the same day, but other tasks were performed in between. In the center-out dataset, trial 752 marks the start of the second block, and in the random dataset, trial 497 marks the start of the second block. In both datasets, some trials are excluded, likely due to trial failure.\

Neural data was originally recorded at 2ksps, and behavior was recorded at 1ksps. However, decoding is typically done on larger time bins. The datasets provided all contain the same data binned at different sizes, 10ms, 20ms, 32ms, 50ms, and 100ms, indicated by the number in the filename. For example, 'r_feats32.pkl' contains the random task data binned at 32ms, and 'r_feats50.pkl' contains the same data binned at 50ms.

## 2. Exploring the data

### 2.1 Setup

First, let's load the data and import things we may need.

In [ ]:
# download dataset


### 2.2 Data Structure

Each .pkl file contains a dictionary of 5 length N numpy.ndarrays, where N is the total number of time bins in the dataset. The keys and their contents are listed below:

* `'neural'` - Nx96 numpy.ndarray containing the binned (average) SBP for all 96 channels.
* `'behavior'` - Nx6 ndarray containing finger kinematics in the following order: `[IDX_position, MRP_position, IDX_velocity, MRP_velocity, IDX_accel, MRP_accel]`
* `'trialNumber'` - Nx1 ndarray containing the current trial number.
* `'targetPosition'` - Nx2 ndarray containing the target position for both finger groups (`[IDX_target, MRP_target]`).
* `'targetWidth'` - Nx1 ndarray containing the target width.